In [5]:
# csce478    Programming Assignment 4     Fall 2020                                   Due: Nov 17 11am
# Linear Support Vector Machine & Principle Component Analysis
# Contributers: Chungsum Kim, Devn Steiner, Jesse Reyes Cortes

#imported packages
import pandas as pd
import numpy as np
import math
import copy
import itertools
from random import sample 
import nltk
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [2]:
# Part A : Linear Support Vector Machine (SVM)

# A.1 implement Linear_SVC model class for performing binary classification

class Linear_SVC:
    def __init__(self, C=1, max_iter=100, tol=None, learning_rate="constant",learning_rate_init=0.001, t_0=1, t_1=1000, early_stopping=False, validation_fraction=0.1, **kwargs):
        print("Linear")
        
    def fit(self, X, Y):
        print("Linear")
        
    def predict(self, X):
        print("Linear")

SyntaxError: invalid syntax (<ipython-input-2-e6772d91a836>, line 7)

In [28]:
# Binary Classification using Linear_SVC Classifier

# A.2 read the iris data, X features: petal length & petal width, recode binary target such that Iris-Virginica is 1 or 0
data = load_iris()

pl = data["data"][:,2]
pw = data["data"][:,3]
X = np.zeros((len(pl), 2))
X[:,0] = pl
X[:,1] = pw

target = np.where(data['target'] == 2, 1, 0)

In [ ]:
# A.3 partition data into train & test set (80% - 20%)
def partition(X, Y, t):     
    test_size = math.ceil(len(Y)*t)
    training_size = len(Y)-test_size
    training_data = X[0:training_size][:]
    testing_data = X[training_size+1:len(Y)] [:]
    training_vector = Y[0:training_size]
    testing_vector = Y[training_size+1:len(Y)]
    
    return training_data, testing_data, training_vector, testing_vector

In [ ]:
# A.4 Hyper-parameter training(kFold - C, learning_rate, max_iter, tol)


In [ ]:
# A.5 train model using optimal values for hyperparameters on test data, report accuracy and test confusion matrix


In [ ]:
# A.6 plot the learning curve


In [ ]:
# A.7 plot the decision boundary and show support vectors using “decision_boundary_support_vectors”
# https://github.com/rhasanbd/Support-Vector-Machine-Classifier-Beginners-Survival-Kit/blob/master/Support%20Vector%20Machine-1-Linearly%20Separable%20Data.ipynb
# Draw a scatter plot

plt.figure(figsize=(16,8))
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bo", label="Class 0")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "ro", label="Class 1")
plt.title("Linearly Separable Data: No Outliers", fontsize=18)
plt.legend(loc=2)
plt.xlabel("$x_1$", fontsize=12)
plt.ylabel("$x_2$", rotation=90, fontsize=12)
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
# A.8 implement early stopping in the "fit" method of the Linear_SVC model, generate early stopping code


In [ ]:
# Part B: Principle Component Analysis

# B.9 using matplotlib.pyploy "imread" function read the image as a 2D matrix


In [ ]:
# B.10 implement steps of eigendecomposition based PCA on X



In [ ]:
# B.11 find top k eigenvectors and create eigen vector matrix using top k eigenvectors


In [ ]:
# B.12 finally project mean centered data on the k top eigenvectors


In [ ]:
# B.13 reconstruct data matrix by taking dot product between projected data and transpose of top K eigenvec matrix


In [ ]:
# B.14 compute reconstruction error between mean centered data matrix X and reconstructed data matrix


In [ ]:
# B.15 perform steps 11-14 for k: 10, 30, 50, 100, 500
# for each k reconstruct image & print the value of k and reconstruction error